# Dask

Dask baut auf bewährten Modulen auf und erweitert diese um Möglichkeiten zur massiven Parallelisierung. So können mehrere NumPy Arrays oder Pandas Dataframes in entsprechenden Dask-Objekten zusammengefasst und für parallele Operationen bereitgestellt werden. Die Dask Objekte stellen große Teile der bakannten API (identisch zu NumPy Arrays oder Pandas Dataframes) bereit.

Dabei kann Dask auch Daten in Objekten ablegen deren größe den verfügbaren Arbeitsspeicher übersteigt. Hierfür lagert Dask teile der Daten in eine verfügbares File System aus. Dask kann daher genutzt werden, um Datenmengen zu verarbeiten, die für Pandas oder NumPy eigentlich zu groß sind. Sind jedoch nur wenige Daten zu verarbeiten/analysieren so kann der für Dask notwendige Overhead zu einer Verlangsamung im Vergleich zu reinen NumPy/Pandas Objekten führen.

![image](images/Dask_Scale.svg)

## Dask Dashboard

Eine Übersicht über die von Dask gestarteten Parallelen Vorgänge und die Auslastung der über Dask reservierten Ressourcen kann über das Dask Dashboard eingesehen werden. Das Client Objekt aus dem dask.distributed Modul ermöglicht das Starten eines Dask Dashboards. Wird das initialisierte Client Objekt ausgegeben, so enthält die Ausgabe eine URL unter der das gestartete Dashboard abgerufen werden kann.

Wenn Jupter in Version 3.0 installiert ist oder zusätzlich Node.js (Version >= 12.0.0) und npm installiert sind, kann alternativ zur manuellen Nutzung auch das dask-labextensions Plugin in Jupyter installiert werden. Dies sorgt für eine Integration des Dask Dashboards in die Jupyter Oberfläche. Am linken Rand ist dann eine neue Schaltfläche "Dask" vorhanden. Über dies kann das Dask Dashboard erreicht werden, ohne dass hierfür eine separate URL aufgerufen werden muss.

In [9]:
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

/home/julian/jupyterlab/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36895 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://94.16.122.16:36895/status,
Dashboard: http://94.16.122.16:36895/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://94.16.122.16/286813/17,Workers: 1
Dashboard: http://94.16.122.16:36895/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://94.16.122.16/286813/20,Total threads: 4
Dashboard: http://94.16.122.16:40727/status,Memory: 1.86 GiB
Nanny: None,


In [10]:
client.close()

Aktuell funktioniert das dask-labextensions Plugin am bwUniCluster noch nicht. Alternativ kann wie oben beschrieben direkt die URL des Dashboards genutzt werden. Hierfür muss über ssh der Port aus der URL aus dem Cluster nach außen weitergeleitet werden. Dies kann mit folgendem Befehl lokal am genutzten Rechner in einer Konsole durchgeführt werden. Der Port und die IP des Jupyter-Compute-Node können dabei der Dashboard-URL entnommen werden.

```bash
ssh -N -L <Port>:<Jupyter-Compute-Node>:<Port> <Hochschulkürzel>_<User-ID>@bwunicluster.scc.kit.edu
```

Nach Ausführen des ssh-Port-Forwardings kann am lokalen Rechner das Dask-Dashboard unter

```bash
http://localhost:<Port>/status
```

aufgerufen werden.

## Dask Array

Dask Array koordiniert mehrere NumPy Arrays und verteilt diese auf die zur Verfügung stehenden Ressourcen. So können Operationen verteilt auf mehrere Threads, Prozesse oder gar Nodes ausgeführt werden. Welche Operationen dabei möglich sind (welche Teile der NumPy Array API auch von Dask Array angeboten werden) kann der Dokumentation entnommen werden: https://docs.dask.org/en/latest/array-api.html.

Weitere Beispiele zu Dask Array: https://mybinder.org/v2/gh/dask/dask-examples/main?urlpath=lab/tree/array.ipynb

## Dask Dataframe

Was Dask Array für NumPy Arrays ist (s.o.), dass ist Dask Dataframe für Pandas Dataframe. Die auf einem Dask Dataframe möglichen Operationen können der Dokumentation entnommen werden: https://docs.dask.org/en/latest/dataframe-api.html.

Eine generelle Übersicht über sinnvolle und weniger sinnvolle Arten einen Dask Dataframe zu nutzen ist unter https://docs.dask.org/en/latest/dataframe.html zu finden.

## Dask und SLURM

Um Dask in Kombination mit SLURM (dem Job-Scheduler des bwUniClusters) nutzen zu können, wird entweder die Klasse SLURMCluster aus dem Modul dask_jobqueue oder das Programm dask-mpi benötigt.

Für die nachfolgenden Übungen wird dask-mpi benötigt. dask_jobqueue ist daher nur grundlegend beschrieben. Eine Umgebung mit allen für die Übungen benötigten Modulen wird nur in der Beschreibung von dask-mpi erstellt.

### dask_jobqueue

WICHTIG: dask_jobqueue setzt eine eins zu eins Beziehung zwischen Job und Node vorraus. D.h. pro Job wird genau ein Node reserviert und genutzt. Werden mehrere Nodes benötigt, so sieht dask_jobqueue vor, dass entsprechend viele Jobs abgesetzt werden. Dies führt auf dem bwUniCluster zu zwei grundlegenden Problemen. Zum einen werden mehrere Jobs unabhängig voneinander gescheduled. Jeder Job hat daher einen eigenen Startzeitpunkt. Es stehen also nur mit viel Glück alle benötigten Ressourcen zeitgleich zur Verfügung. Zum anderen sehen alle multiple-Queues vor, dass pro Job mindestens zwei Nodes reserviert werden. Diese Queues sind explizit für Anwendungsfälle gedacht in denen mehrere Nodes zur gleichen Zeit benötigt werden. Werden diese Queues zusammen mit dask_jobqueue genutzt, so setzt dask_jobqueue für jeden benötigten Node einen eigenen Job ab, der jeweils zwei Nodes reserviert, von denen dask_jobqueue aber dann nur einen nutzt.

Fazit:
- um mehrere Nodes zeitgleich zur Verfügung zu haben, sieht bwUniCluster einen Job in einer multiple-Queue mit mehreren Nodes pro Job vor
- dask_jobqueue setzt vorraus, dass pro Job nur ein Node genutzt wird
- anstelle von dask_jobqueue sollte dask-mpi genutzt werden


Damit dask_jobqueue zur Verfügung steht muss im jeweiligen Environment sowohl dask als auch dask_jobqueue installiert sein:

```bash
python3 -m pip install dask_jobqueue dask
```

Ist der IPython-Kernel aus einem entsprechend erweiterten Environment im Jupyter registriert, so kann dieser beim Start eines neuen Notebooks ausgewählt werden. Anschließend kann die SLURMCluster Klasse im Notebook importiert und zum Erstellen einer SLURM-Job-Konfiguration genutzt werden.

Welche queues für eine solche Konfiguration am bwUniCluster zur Verfügung stehen und welche Eigenschaften diese Haben kann der Dokumentation unter

https://wiki.bwhpc.de/e/BwUniCluster_2.0_Batch_Queues

entnommen werden.

In [2]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    queue='multiple', # queue multiple ermöglicht eine Reservierung von mehreren Nodes pro Job (min. 2 Nodes pro Job)
    cores=40, # ein Node der queue multiple besitzt 40 cores => für 80 cores werden zwei Nodes angefordert
    memory="90GB", # maximal verfügbarer Speicher pro Node in queue multiple
    local_directory='/tmp', # Daten sollen lokal im Node und nicht über Netzwerk ins zentrale Filesystem geschrieben werden
    walltime='00:30:00', # Nodes sollen eine halbe Stunde reserviert werden
    interface='ib0' # für die Netzwerkkommunikation im Cluster wollen wir schnelles Infiniband nutzen
)

Der eigentliche Job wird dann auf Basis der Konfiguration mittels der Methode scale gestartet:

In [3]:
cluster.scale(jobs=1) # beim Start der Konfiguration können auch mehrere Jobs gleichzeitig gestartet werden (hierdurch ist das Reservieren mehrerer Nodes möglich)

In [4]:
from dask.distributed import Client
client = Client(cluster)
client # enthält die Informationen über den gestarteten Cluster

<Client: 'tcp://172.26.21.158:44599' processes=10 threads=80, memory=83.80 GiB>

### dask-mpi

Die Anwendung dask-mpi ermöglicht das Starten eines dask-Clusters über MPI. Hierdurch ist es möglich, mehrere Nodes mit nur einem Job zu reservieren. Dadurch können auch Queues genutzt werden, die mehr als einen Node pro Job voraussetzen. Zudem ist sichergestellt, dass alle benötigten Nodes zum gleichen Zeitpunkt verfügbar sind (da sie über den gleichen Job angefordert wurden).

#### Environment 

Um dask-mpi mit allen für die Übungen benötigten Modulen verfügbar zu machen, muss zunächst ein entsprechendes Environment erstellt werden. Hierfür nutzen wir Miniconda.

Miniconda bietet die Möglichkeit ein Environment über das Laden von fertigen Binaries zu erstellen. Im Gegensatz zu pip install reduziert dies die nötigen Abhängigkeiten, da bei pip install gegebenenfalls Software gebaut wird (z.B. mpi für dask-mpi) und hierfür alle zum Bauen/Kompilieren notwendigen Komponenten in der jeweils passenden Form vorhanden sein müssen.

Zunächst benötigen wir eine aktuelle Version von Miniconda (der folgende Befehl muss wie alle nachfolgenden Befehle im Terminal File->New->Terminal ausgeführt werden):

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
```

Anschließend muss das geladene sh-Script mit den zum Ausführen nötigen Rechten ausgestattet und dann ausgeführt werden:

```bash
chmod +x Miniconda3-latest-Linux-x86_64.sh
```

```bash
./Miniconda3-latest-Linux-x86_64.sh
```

Während der Ausführung des Scripts kommt zunächst die Abfrage "Please, press ENTER to continue". Diese bitte mit der Enter-Taste bestätigen.

Anschließend bitte den Lizenzvereinbarungen zustimmen. Mittels der Leertaste kann zum Ende der Vereinbarung gesprungen werden. Dannach erscheint die Abfrage "Please answer 'yes' or 'no':". Durch die Eingabe von "yes" und Betätigen der Enter-Taste kann fortgefahren werden.

Bei der Abfrage "Press ENTER to confirm the location" benötigen wir die Standardeinstellung. Dementsprechend bitte einfach mit der Enter-Taste bestätigen.

Die Abfrage "Do you wish the installer to initialize Miniconda3 by running conda init? \[yes|no\]" bitte mit "yes" bestätigen.

Nachdem die Installation von Miniconda abgeschlossen ist, ist das bei der Installation erstellte base-Environment in die bashrc-Datei eingetragen. Dies sorgt dafür, dass dieses Environment beim Starten eines Terminals automatisch aktiviert wird. Wir empfehlen diesen Automatismus über den folgenden Befehl zu deaktivieren:

```bash
conda config --set auto_activate_base false
```

Das für die folgenden Übungen benötigte Environment wird mit dem folgenden Befehl erstellt:

```bash
conda create -n python_workshop_env python=3
```

Die Abfrage "Proceed (\[y\]/n)?" bitte mit "y" bestätigen.

Um alle Änderungen im aktuellen Terminal zu aktivieren, muss die bashrc ausgeführt werden. Der "." zu Beginn des Befehls ist die Kurzform von "source".

```bash
. ~/.bashrc
```

Damit die nachfolgenden Installationen in das neu angelegte Environment installieren, muss dieses aktiviert werden:

```bash
conda activate python_workshop_env
```

Für dask-mpi werden die Pakete dask, "dask[distributed]", bokeh, ipykernel und mpi4py benötigt. Für die Übungen zusätzlich die Pakete s3fs, numpy und pandas TODO: weitere Pakete ?????. Der nachfolgende Befehl installiert diese in das Environment:

```bash
conda install s3fs bokeh dask ipykernel numpy pandas "dask[distributed]" mpi4py
```

Die Abfrage "Proceed (\[y\]/n)?" bitte mit "y" bestätigen.

Abschließend muss noch dask-mpi installiert werden:

```bash
conda install -c conda-forge dask-mpi
```

Um das neu erstellte Environment auf die genutzten Nodes zu kopieren, sollte dieses in ein Archiv gepackt werden:

```bash
tar -zcvf ~/miniconda3/envs/python_workshop_env.gz -C ~/miniconda3/envs/ python_workshop_env/
```

#### sbatch und Job-Script

Zum Starten eines dask-Clusters über dask-mpi wird ein Job-Script benötigt. Dieses wird später über sbatch gestartet. Sbatch sorgt dafür, dass die benötigte Anzahl an Knoten reserviert wird (dabei wird auch festgelegt, wieviele Tasks gestartet werden sollen und wie diese über die Nodes verteilt werden sollen). Nachdem die angeforderten Knoten verfügbar sind, führt sbatch das übergebene Script auf dem ersten der reservierten Nodes aus. In diesem Script können dann benötigte Daten auf die einzelnen Nodes kopiert werden (über separate Jobs, welche per srun gestartet werden; lokale Daten können im Vergleich zu Zugriffen auf das zentrale File System schneller eingelesen werden). Abschließend führt das Script einen Aufruf von mpirun aus. Über mpirun wird dask-mpi n mal gestartet. Dabei sollte n gleich der Anzahl der benötigten Worker plus eins (für den Scheduler) sein. Die Anzahl n muss dabei zur Anzahl der über sbatch konfigurierten Tasks passen.

#### Anzahl Prozesse pro Node und Anzahl Threads pro Prozess:

Pro Node sollte die Anzahl der Threads pro Prozess multipliziert mit der Anzahl der Prozesse der Summe der Verfügbaren Cores entsprechen. Nur so ist gewährleistet, dass alle Threads einen Core verfügbar haben und nicht durch andere Threads ausgebremst werden.

Informationen über die Queues: https://wiki.bwhpc.de/e/BwUniCluster_2.0_Batch_Queues

Informationen über die Hardware pro Node: https://wiki.bwhpc.de/e/BwUniCluster_2.0_Hardware_and_Architecture

Bei überwiegender Nutzung von numerischen Bibliotheken ist eine hohe Anzahl von Threads pro Prozess sinnvoll, da diese Bibliotheken (Numpy, Pandas, ...) so implementiert sind, dass sie nicht vom Global Interpreter Lock (GIL) von Python ausgebremst werden. Sie sind meist in C geschrieben und bieten nur ein Interface für Zugriffe aus Python. Gleichzeitig profitieren sie von der gemeinsamen Nutzung der selben Daten innerhalb eines Prozesses (keine Interprozesskommunikation zwischen Threads im gleichen Prozess nötig).

Werden hauptsächlich in Python geschriebene Algorithmen ausgeführt, so bieten sich viele Prozess mit wenigen oder gar nur je einem Thread pro Prozess an, da das GIL hier eine parallele Ausführung mit mehreren Threads in einem Prozess verhindert.

Mischungen von numerischen Bibliotheken und in Python implementierten Algorithmen machen ein ausgewogeneres Verhältnis zwischen Prozessen und Threads notwendig.

In [43]:
import os

count_nodes = 2
count_worker_per_node = 4 # auf dem ersten Node wird noch der Scheduler gestartet => der erste Node stellt einen Worker weniger zur Verfügung
count_threads_per_worker = 10
time = "30:00" # 30 min.
mem = "90000mb"

f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w") # ein neues Script erstellen, welches dann per sbatch abgeschickt werden kann
f.write(f"""#!/bin/bash -l
# -l dient dazu die Einstellungen der bashrc zu übernehmen (wird für conda benötigt)

# jeden über sbatch reservierten Node einzeln in einer for-Schleife durchgehen
NODES=$(scontrol show hostname | cat)
for NODE in $NODES
do
    # pro Node einen Job per srun starten und Daten unter /tmp lokal im Node speichern
    # das "&" am ende sorgt dafür, dass der srun parallel im Hintergrund ausgeführt wird
    # der "\" am Zeilenende maskiert das Zielenende (nächste Zeile gehört noch zur aktuellen Zeile)
    # mkdir erstellt ein Verzeichnis
    # cp kopiert eine Datei
    # tar entpackt die Datei
    srun -N 1 -n 1 -w $NODE --pty /bin/bash -c "mkdir /tmp/envs \
    && cp ~/miniconda3/envs/python_workshop_env.gz /tmp/envs && \
    tar -zxvf /tmp/envs/python_workshop_env.gz --directory /tmp/envs" &
done

# warten auf alle parallel gestarteten Befehle
wait

# Aktivieren des lokalen Environments (muss nicht auf jedem Node durchgeführt werden, da die entsprechenden Umgebungsvariablen per mpirun übernommen werden)
conda activate /tmp/envs/python_workshop_env

# startet die Worker und einen Scheduler auf den Nodes
# -np x: x = Anzahl Worker + 1 für Scheduler (x muss gleich --ntasks bei Ausführung von sbatch sein)
# --scheduler-file: sobald alle Worker gestartet sind, schreibt der Scheduler in diese Datei die Verbindungsinformationen des dask-Clusters)
# --interface='ib0': für Kommunikation zwischen den Nodes nutzen wir Infiniband
# --local-directory='/tmp': wenn dask aufgrund fehlendem Arbeitsspeicher Daten auslagern muss, soll hierfür das lokale Dateisystem des Nodes genutzt werden
# --no-nanny: keinen überwachenden Nanny-Prozess pro Worker Prozess
# --map-by core:PE=n: jedem Prozess sollen n cores zugewiesen werden
mpirun --map-by core:PE={count_threads_per_worker} -np {count_nodes * count_worker_per_node} dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp' --no-nanny --nthreads={count_threads_per_worker} --name base""")
f.close()

In [44]:
# wie viele Nodes sind pro Queue aktuell frei verfügbar
# kann auch in einem Terminal (File->New->Terminal) außerhalb des Jupyter Notebooks als Befehl "sinfo_t_idle" abgesetzt werden
os.system("sinfo_t_idle")

Partition dev_single    :      5 nodes idle
Partition single        :     24 nodes idle
Partition dev_multiple  :      8 nodes idle
Partition multiple      :     64 nodes idle
Partition fat           :      2 nodes idle
Partition dev_multiple_e:      8 nodes idle
Partition multiple_e    :      1 nodes idle
Partition dev_special   :      2 nodes idle
Partition special       :      0 nodes idle
Partition gpu_4         :      0 nodes idle
Partition dev_gpu_4     :      1 nodes idle
Partition gpu_8         :      0 nodes idle


0

In [45]:
queue = "dev_multiple"

# rm: Löschen des ~/dask-scheduler.json Files: sobald das File wieder da ist, ist der angeforderte Dask-Cluster vollständig gestartet
# sbatch: reserviert die notwendigen Ressourcen (Nodes) in der angegebenen queue und startet das Script ~/job_dask_mpi.sh auf dem ersten der Nodes
os.system(f"rm -f ~/dask-scheduler.json && sbatch -p {queue} --nodes={count_nodes} --ntasks={count_nodes * count_worker_per_node} --ntasks-per-node={count_worker_per_node} --time={time} --mem={mem} ~/job_dask_mpi.sh")

Submitted batch job 20152366


0

In [46]:
# steht bereits ein Zeitpunkt fest, an dem die Ressourcen für uns verfügbar sind?
os.system("squeue --start")

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)
          20152366 dev_multi job_dask es_pkoes PD                 N/A      2 (null)               (Priority)


0

In [47]:
# welche Ressourcen sind für unseren User angefordert?
# Zeigt eine Zeile pro Job mit dem Ressourcen angefordert wurden.
# ST: Status
#     PD: Pending, Ressourcen wurden angefordert, stehen aber noch nicht zur Verfügung
#     R:  Running, Ressourcen sind verfügbar
#     CG: Completing, Job ist beendet/abgebrochen, allerdings laufen noch einzelne Prozesse (die noch beendet oder abgebrochen werden)
#     weitere Status-Codes: https://curc.readthedocs.io/en/latest/running-jobs/squeue-status-codes.html
# TIME: wie lange sind die Ressourcen schon durch uns genutzt
# NODES: Anzahl der Reservierten Nodes
# NODELIST: welche Nodes sind Reserviert
#     Der Namen der Nodes ist mit einem Prefix (ist für jeden Node gleich) gefolgt von der Nummerierung der einzelnen Nodes in Eckigen Klammern angegeben.
#     Ein Bindestrich zwischen zwei Nummern in den Eckigen Klammern bedeutet, dass alle Nummern zwischen den beiden angegebenen für uns Reserviert sind.
#     Ein Komma zwischen zwei Nummern bedeutet, dass die beiden Nummern für uns Reserviert sind.
#     Beispiele:
#         uc2n[001-003] zeigt an, dass uc2n001, uc2n002 und uc2n003 reserviert wurden
#         uc2n[001,003] zeigt an, dass uc2n001 und uc2n003 reserviert wurden
os.system("squeue")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          20152366 dev_multi job_dask es_pkoes PD       0:00      2 (Priority)
          20151897    single jupyterh es_pkoes  R    2:20:40      1 uc2n366


0

In [36]:
import os, time

# warten, bis die Datei ~/dask-scheduler.json vorhanden ist (diese wird vom Dask-Scheduler geschrieben, sobald alle Worker gestartet und bereit sind)
while not os.path.isfile(os.path.expanduser("~/dask-scheduler.json")):
    time.sleep(1)

In [37]:
# Über ein Client-Objekt können wir das aktuelle Jupyter-Notebook mit dem erstellten Dask-Cluster verbinden.
# Dafür nutzen wir die ~/dask-scheduler.json Datei. In dieser sind IP und Port des Schedulers gespeichert.
# (anstatt aus einem Jupter-Notebook heraus, kann dies z.B. auch aus einem Python-Script heraus geschehen)
from dask.distributed import Client
client = Client(scheduler_file=os.path.expanduser('~/dask-scheduler.json'))

In [38]:
# wir lassen den Client anzeigen (dieser enthält auch Informationen zum Scheduler und den einzelnen Workern)
client

<Client: 'tcp://172.26.20.1:33587' processes=7 threads=70, memory=76.90 GiB>

Weitere Worker können mit einem separaten Job erstellt und dem Scheduler aus einem vorhergehenden Job hinzugefügt werden. Hierzu wird auch die scheduler.json-Datei genutzt.

```bash
mpirun -n 5 dask-mpi --scheduler-file scheduler.json --no-scheduler
```

In [39]:
count_nodes = 2
count_worker_per_node = 4
count_threads_per_worker = 10
time = "30:00" # 30 min.
mem = "90000mb"

# ein Script um einen existierenden Cluster um einen Node zu erweitern
f = open(os.path.expanduser("~/job_dask_mpi_2.sh"), "w")
f.write(f"""#!/bin/bash -l
NODES=$(scontrol show hostname | cat)
for NODE in $NODES
do
    srun -N 1 -n 1 -w $NODE --pty /bin/bash -c "mkdir /tmp/envs \
    && cp ~/miniconda3/envs/python_workshop_env.gz /tmp/envs && \
    tar -zxvf /tmp/envs/python_workshop_env.gz --directory /tmp/envs" &
done

wait

conda activate /tmp/envs/python_workshop_env

# da ein existierender Dask-Cluster erweitert werden soll:
# --no-scheduler: erstelle keinen eigenen Scheduler, sondern registriere die Worker an dem im scheduler-file angegebenen Scheduler
# --name
mpirun --map-by core:PE={count_threads_per_worker} -np {count_nodes * count_worker_per_node} dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp' --no-nanny --nthreads={count_threads_per_worker} --no-scheduler --name expansion""")
f.close()

# einen Node über sbatch reservieren und das Script auf diesem ausführen
os.system(f"sbatch -p {queue} --nodes={count_nodes} --ntasks={count_nodes * count_worker_per_node} --ntasks-per-node={count_worker_per_node} --time={time} --mem={mem} ~/job_dask_mpi_2.sh")

Submitted batch job 20152349


0

In [40]:
os.system("squeue")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          20152349 dev_multi job_dask es_pkoes PD       0:00      2 (Priority)
          20152343 dev_multi job_dask es_pkoes  R       3:59      2 uc2n[001-002]
          20151897    single jupyterh es_pkoes  R    2:09:30      1 uc2n366


0

In [41]:
client.wait_for_workers(15)

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


OSError: Timed out trying to connect to tcp://172.26.20.1:33587 after 30 s

In [30]:
client

<Client: 'tcp://172.26.20.1:37837' processes=8 threads=80, memory=87.89 GiB>

In [ ]:
job_id = ????????
os.system(f"scancel {job_id}")

In [ ]:
from distributed import Client
client = Client('10.0.1.162:40675')

In [ ]:
client

In [ ]:
import dask.array as da
x = da.random.random((100000,100000), chunks="16 MiB")
x

In [ ]:
y = (x + x.T) - x.mean(axis=0)

In [ ]:
y.sum().compute()